In [9]:
import tkinter as tk
from math import sin, cos, radians
import random

class AirplaneSimulator:
    def __init__(self, root):
        self.root = root
        self.root.title("Airplane Flight Simulator")
        self.root.geometry("800x600")
        self.root.configure(bg='#1a1a2e')
        
        # Flight parameters
        self.altitude = 5000  # feet
        self.speed = 250  # knots
        self.pitch = 0  # degrees (-90 to 90)
        self.roll = 0  # degrees (-180 to 180)
        self.heading = 0  # degrees (0-360)
        self.throttle = 50  # percentage
        self.vertical_speed = 0  # feet per minute
        
        # Physics
        self.gravity = 0.5
        self.running = True
        
        # Create UI
        self.create_widgets()
        
        # Start simulation
        self.update_simulation()
        
    def create_widgets(self):
        # Main canvas for artificial horizon
        self.canvas = tk.Canvas(self.root, width=600, height=400, bg='#0f3460', highlightthickness=0)
        self.canvas.pack(pady=10)
        
        # Instrument panel
        panel = tk.Frame(self.root, bg='#1a1a2e')
        panel.pack(fill=tk.X, padx=20)
        
        # Create instruments
        self.create_instrument(panel, "Altitude", "altitude_label", 0)
        self.create_instrument(panel, "Speed", "speed_label", 1)
        self.create_instrument(panel, "V/S", "vs_label", 2)
        self.create_instrument(panel, "Heading", "heading_label", 3)
        
        # Control panel
        controls = tk.Frame(self.root, bg='#1a1a2e')
        controls.pack(pady=10)
        
        tk.Label(controls, text="Throttle:", fg='white', bg='#1a1a2e').grid(row=0, column=0, padx=5)
        self.throttle_scale = tk.Scale(controls, from_=0, to=100, orient=tk.HORIZONTAL, 
                                       command=self.update_throttle, bg='#16213e', fg='white',
                                       highlightthickness=0, length=200)
        self.throttle_scale.set(50)
        self.throttle_scale.grid(row=0, column=1, padx=5)
        
        # Instructions
        instructions = tk.Label(self.root, 
                               text="Controls: ↑↓ = Pitch | ←→ = Roll | Q/E = Heading | Throttle Slider",
                               fg='#e94560', bg='#1a1a2e', font=('Arial', 10))
        instructions.pack()
        
        # Bind keys
        self.root.bind('<Up>', lambda e: self.adjust_pitch(2))
        self.root.bind('<Down>', lambda e: self.adjust_pitch(-2))
        self.root.bind('<Left>', lambda e: self.adjust_roll(-3))
        self.root.bind('<Right>', lambda e: self.adjust_roll(3))
        self.root.bind('q', lambda e: self.adjust_heading(-2))
        self.root.bind('e', lambda e: self.adjust_heading(2))
        
    def create_instrument(self, parent, label, attr_name, col):
        frame = tk.Frame(parent, bg='#16213e', relief=tk.RIDGE, borderwidth=2)
        frame.grid(row=0, column=col, padx=10, pady=5)
        
        tk.Label(frame, text=label, fg='#e94560', bg='#16213e', 
                font=('Arial', 10, 'bold')).pack()
        label_widget = tk.Label(frame, text="0", fg='white', bg='#16213e', 
                               font=('Arial', 16), width=8)
        label_widget.pack(pady=5)
        setattr(self, attr_name, label_widget)
        
    def draw_horizon(self):
        self.canvas.delete('all')
        w, h = 600, 400
        cx, cy = w // 2, h // 2
        
        # Calculate horizon position based on pitch
        pitch_offset = self.pitch * 3
        
        # Sky gradient
        sky_y = cy - pitch_offset
        self.canvas.create_rectangle(0, 0, w, sky_y, fill='#4a9eff', outline='')
        
        # Ground
        self.canvas.create_rectangle(0, sky_y, w, h, fill='#8b4513', outline='')
        
        # Horizon line (with roll)
        angle = radians(self.roll)
        length = 800
        x1 = cx - length * cos(angle)
        y1 = sky_y - length * sin(angle)
        x2 = cx + length * cos(angle)
        y2 = sky_y + length * sin(angle)
        self.canvas.create_line(x1, y1, x2, y2, fill='white', width=3)
        
        # Pitch ladder
        for i in range(-30, 40, 10):
            if i == 0:
                continue
            y_pos = cy - pitch_offset - i * 3
            ladder_len = 40 if i % 20 == 0 else 25
            
            # Rotate ladder lines with roll
            dx = ladder_len * cos(angle)
            dy = ladder_len * sin(angle)
            
            self.canvas.create_line(cx - dx, y_pos - dy, cx + dx, y_pos + dy, 
                                   fill='white', width=2)
            self.canvas.create_text(cx + dx + 20, y_pos + dy, text=f"{i}°", 
                                   fill='white', font=('Arial', 8))
        
        # Aircraft symbol (fixed in center)
        wing_size = 60
        self.canvas.create_line(cx - wing_size, cy, cx - 15, cy, fill='yellow', width=4)
        self.canvas.create_line(cx + 15, cy, cx + wing_size, cy, fill='yellow', width=4)
        self.canvas.create_oval(cx - 8, cy - 8, cx + 8, cy + 8, fill='yellow', outline='')
        
        # Roll indicator
        self.canvas.create_arc(cx - 80, 30, cx + 80, 190, start=0, extent=180, 
                              outline='white', width=2, style=tk.ARC)
        # Roll pointer
        pointer_x = cx + 70 * sin(angle)
        pointer_y = 110 - 70 * cos(angle)
        self.canvas.create_polygon(pointer_x, pointer_y,
                                   pointer_x - 5, pointer_y + 10,
                                   pointer_x + 5, pointer_y + 10,
                                   fill='yellow')
        
    def update_throttle(self, val):
        self.throttle = int(val)
        
    def adjust_pitch(self, delta):
        self.pitch = max(-30, min(30, self.pitch + delta))
        
    def adjust_roll(self, delta):
        self.roll = (self.roll + delta) % 360
        if self.roll > 180:
            self.roll -= 360
            
    def adjust_heading(self, delta):
        self.heading = (self.heading + delta) % 360
        
    def update_physics(self):
        # Speed changes based on throttle and pitch
        target_speed = self.throttle * 3 + 50
        speed_diff = target_speed - self.speed
        self.speed += speed_diff * 0.02
        
        # Vertical speed based on pitch and speed
        pitch_effect = self.pitch * 50
        self.vertical_speed = pitch_effect * (self.speed / 250)
        
        # Apply gravity effect
        if self.pitch < 0:
            self.vertical_speed -= self.gravity * 10
            
        # Update altitude
        self.altitude += self.vertical_speed / 60  # Convert to feet per frame
        
        # Ground collision
        if self.altitude < 0:
            self.altitude = 0
            self.vertical_speed = 0
            
        # Roll returns to level gradually
        if abs(self.roll) > 1:
            self.roll *= 0.98
            
    def update_instruments(self):
        self.altitude_label.config(text=f"{int(self.altitude)} ft")
        self.speed_label.config(text=f"{int(self.speed)} kts")
        self.vs_label.config(text=f"{int(self.vertical_speed):+d} fpm")
        self.heading_label.config(text=f"{int(self.heading)}°")
        
    def update_simulation(self):
        if self.running:
            self.update_physics()
            self.draw_horizon()
            self.update_instruments()
            self.root.after(33, self.update_simulation)  # ~30 FPS
            
if __name__ == "__main__":
    root = tk.Tk()
    sim = AirplaneSimulator(root)
    root.mainloop()

In [10]:
import pygame
import sys
import random

# Initialize pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 800, 600
PADDLE_WIDTH, PADDLE_HEIGHT = 15, 100
BALL_SIZE = 15
FPS = 60
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (0, 120, 255)
RED = (255, 50, 50)
GREEN = (50, 255, 100)
FONT_SIZE = 36

# Create the game window
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Professional Ping Pong")
clock = pygame.time.Clock()

# Game objects
player_paddle = pygame.Rect(50, HEIGHT // 2 - PADDLE_HEIGHT // 2, PADDLE_WIDTH, PADDLE_HEIGHT)
opponent_paddle = pygame.Rect(WIDTH - 50 - PADDLE_WIDTH, HEIGHT // 2 - PADDLE_HEIGHT // 2, PADDLE_WIDTH, PADDLE_HEIGHT)
ball = pygame.Rect(WIDTH // 2 - BALL_SIZE // 2, HEIGHT // 2 - BALL_SIZE // 2, BALL_SIZE, BALL_SIZE)

# Game variables
player_speed = 0
opponent_speed = 7
ball_speed_x = 7 * random.choice((1, -1))
ball_speed_y = 7 * random.choice((1, -1))
player_score = 0
opponent_score = 0
game_font = pygame.font.Font(None, FONT_SIZE)

def reset_ball():
    """Reset the ball to the center with a random direction"""
    ball.center = (WIDTH // 2, HEIGHT // 2)
    return 7 * random.choice((1, -1)), 7 * random.choice((1, -1))

def draw_objects():
    """Draw all game objects on the screen"""
    # Fill background
    screen.fill(BLACK)
    
    # Draw center line
    pygame.draw.aaline(screen, WHITE, (WIDTH // 2, 0), (WIDTH // 2, HEIGHT))
    
    # Draw paddles and ball
    pygame.draw.rect(screen, BLUE, player_paddle)
    pygame.draw.rect(screen, RED, opponent_paddle)
    pygame.draw.ellipse(screen, GREEN, ball)
    
    # Draw scores
    player_text = game_font.render(f"{player_score}", False, WHITE)
    opponent_text = game_font.render(f"{opponent_score}", False, WHITE)
    screen.blit(player_text, (WIDTH // 4, 20))
    screen.blit(opponent_text, (3 * WIDTH // 4 - opponent_text.get_width(), 20))

def move_paddles():
    """Move player and opponent paddles"""
    # Player paddle movement
    player_paddle.y += player_speed
    
    # Keep player paddle on screen
    if player_paddle.top <= 0:
        player_paddle.top = 0
    if player_paddle.bottom >= HEIGHT:
        player_paddle.bottom = HEIGHT
    
    # Opponent AI - follow the ball
    if opponent_paddle.centery < ball.centery:
        opponent_paddle.y += opponent_speed
    elif opponent_paddle.centery > ball.centery:
        opponent_paddle.y -= opponent_speed
    
    # Keep opponent paddle on screen
    if opponent_paddle.top <= 0:
        opponent_paddle.top = 0
    if opponent_paddle.bottom >= HEIGHT:
        opponent_paddle.bottom = HEIGHT

def move_ball():
    """Move the ball and handle collisions"""
    global ball_speed_x, ball_speed_y, player_score, opponent_score
    
    # Move the ball
    ball.x += ball_speed_x
    ball.y += ball_speed_y
    
    # Ball collision with top and bottom
    if ball.top <= 0 or ball.bottom >= HEIGHT:
        ball_speed_y *= -1
        # Add a slight angle variation on wall bounce
        ball_speed_y += random.uniform(-0.5, 0.5)
    
    # Ball collision with paddles
    if ball.colliderect(player_paddle) or ball.colliderect(opponent_paddle):
        ball_speed_x *= -1
        # Add a slight angle variation based on where the ball hits the paddle
        if ball.colliderect(player_paddle):
            ball_speed_x += 0.5  # Increase speed slightly
            # Adjust angle based on where ball hits paddle
            relative_intersect_y = (player_paddle.centery - ball.centery) / (PADDLE_HEIGHT / 2)
            ball_speed_y = -relative_intersect_y * 5
        else:
            ball_speed_x -= 0.5  # Increase speed slightly
            # Adjust angle based on where ball hits paddle
            relative_intersect_y = (opponent_paddle.centery - ball.centery) / (PADDLE_HEIGHT / 2)
            ball_speed_y = -relative_intersect_y * 5
        
        # Limit maximum speed
        ball_speed_x = max(min(ball_speed_x, 12), -12)
        ball_speed_y = max(min(ball_speed_y, 12), -12)
    
    # Ball goes out of bounds
    if ball.left <= 0:
        opponent_score += 1
        ball_speed_x, ball_speed_y = reset_ball()
    if ball.right >= WIDTH:
        player_score += 1
        ball_speed_x, ball_speed_y = reset_ball()

# Main game loop
while True:
    # Handle events
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        
        # Player paddle controls
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP:
                player_speed = -7
            if event.key == pygame.K_DOWN:
                player_speed = 7
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_UP or event.key == pygame.K_DOWN:
                player_speed = 0
    
    # Game logic
    move_paddles()
    move_ball()
    
    # Drawing
    draw_objects()
    
    # Update display
    pygame.display.flip()
    clock.tick(FPS)

ModuleNotFoundError: No module named 'pygame'

In [ ]:
import asyncio
import os
from aiogram import Bot, Dispatcher, types
from aiogram.filters import CommandStart
from aiogram.types import InlineKeyboardMarkup, InlineKeyboardButton, CallbackQuery
import asyncpg
import random

# Simple in-memory storage for active users by language
users = {}  # {user_id: partner_id}
queues = {
    'uz': [],  # O'zbek tilida kutayotgan foydalanuvchilar
    'ru': [],  # Rus tilida kutayotgan foydalanuvchilar  
    'en': []   # Ingliz tilida kutayotgan foydalanuvchilar
}
user_languages = {}  # {user_id: 'uz'/'ru'/'en'}

bot = Bot(os.getenv("BOT_TOKEN"))
dp = Dispatcher()

# Translations
texts = {
    'uz': {
        'welcome': "🌍 *Anonim Chatga xush kelibsiz!*\n\nDunyo bo'ylab odamlar bilan anonim suhbatlashing.\nTilni tanlang:",
        'language_selected': "✅ Til tanlandi! Endi xabar yuboring va suhbatni boshlang!\n\nKomandalar:\n/next - Yangi sherik topish\n/stop - Suhbatni to'xtatish\n/language - Tilni o'zgartirish",
        'searching': "🔍 Sherik qidirilmoqda...",
        'connected': "💚 Ulandi! Anonim suhbatni boshlang!",
        'partner_left': "💔 Sherik ketdi. Yangi sherik topish uchun xabar yuboring!",
        'chat_stopped': "❌ Suhbat to'xtatildi. Yangi sherik topish uchun xabar yuboring!",
        'not_in_chat': "❌ Siz hozir suhbatda emassiz. Boshlash uchun xabar yuboring!",
        'select_language': "Tilni tanlang:"
    },
    'ru': {
        'welcome': "🌍 *Добро пожаловать в Анонимный Чат!*\n\nОбщайтесь с людьми по всему миру анонимно.\nВыберите язык:",
        'language_selected': "✅ Язык выбран! Теперь отправьте сообщение и начните общение!\n\nКоманды:\n/next - Найти нового собеседника\n/stop - Остановить чат\n/language - Изменить язык",
        'searching': "🔍 Поиск собеседника...",
        'connected': "💚 Подключен! Начните анонимное общение!",
        'partner_left': "💔 Собеседник ушел. Отправьте сообщение, чтобы найти нового!",
        'chat_stopped': "❌ Чат остановлен. Отправьте сообщение, чтобы найти нового собеседника!",
        'not_in_chat': "❌ Вы не в чате. Отправьте сообщение, чтобы начать!",
        'select_language': "Выберите язык:"
    },
    'en': {
        'welcome': "🌍 *Welcome to Anonymous Chat!*\n\nConnect with random people worldwide anonymously.\nSelect your language:",
        'language_selected': "✅ Language selected! Now send any message to start chatting!\n\nCommands:\n/next - Find new partner\n/stop - Stop current chat\n/language - Change language",
        'searching': "🔍 Searching for partner...",
        'connected': "💚 Connected! Start chatting anonymously!",
        'partner_left': "💔 Partner left. Send any message to find new partner!",
        'chat_stopped': "❌ Chat stopped. Send any message to find new partner!",
        'not_in_chat': "❌ You're not in any chat. Send any message to start!",
        'select_language': "Select language:"
    }
}

def get_text(user_id, key):
    lang = user_languages.get(user_id, 'en')
    return texts[lang][key]

def get_language_keyboard():
    keyboard = InlineKeyboardMarkup(inline_keyboard=[
        [
            InlineKeyboardButton(text="🇺🇿 O'zbek", callback_data="lang_uz"),
            InlineKeyboardButton(text="🇷🇺 Русский", callback_data="lang_ru")
        ],
        [
            InlineKeyboardButton(text="🇺🇸 English", callback_data="lang_en")
        ]
    ])
    return keyboard

# Database connection
async def get_db():
    return await asyncpg.connect(os.getenv("DATABASE_URL"))

@dp.message(CommandStart())
async def start(message: types.Message):
    user_id = message.from_user.id
    
    # Check if user already exists in database
    try:
        conn = await get_db()
        user_record = await conn.fetchrow("SELECT * FROM users WHERE user_id = $1", user_id)
        
        if user_record:
            # User exists, load their language
            user_languages[user_id] = user_record['language']
            await message.answer(
                get_text(user_id, 'language_selected'),
                parse_mode="Markdown"
            )
        else:
            # New user, show language selection
            await message.answer(
                texts['en']['welcome'],  # Default to English for welcome
                reply_markup=get_language_keyboard(),
                parse_mode="Markdown"
            )
        
        await conn.close()
    except Exception as e:
        print(f"Database error: {e}")
        # Fallback without database
        await message.answer(
            texts['en']['welcome'],
            reply_markup=get_language_keyboard(),
            parse_mode="Markdown"
        )

@dp.callback_query(lambda c: c.data.startswith('lang_'))
async def language_callback(callback: CallbackQuery):
    user_id = callback.from_user.id
    language = callback.data.split('_')[1]  # uz, ru, en
    
    # Save user language
    user_languages[user_id] = language
    
    # Save to database
    try:
        conn = await get_db()
        await conn.execute("""
            INSERT INTO users (user_id, language, username, first_name) 
            VALUES ($1, $2, $3, $4) 
            ON CONFLICT (user_id) 
            DO UPDATE SET language = $2
        """, user_id, language, callback.from_user.username, callback.from_user.first_name)
        await conn.close()
    except Exception as e:
        print(f"Database error: {e}")
    
    await callback.message.edit_text(
        get_text(user_id, 'language_selected'),
        parse_mode="Markdown"
    )
    await callback.answer()

@dp.message(lambda message: message.text == "/language")
async def change_language(message: types.Message):
    user_id = message.from_user.id
    current_lang = user_languages.get(user_id, 'en')
    
    await message.answer(
        texts[current_lang]['select_language'],
        reply_markup=get_language_keyboard()
    )

@dp.message(lambda message: message.text == "/next")
async def next_partner(message: types.Message):
    user_id = message.from_user.id
    
    if user_id not in user_languages:
        await message.answer(
            texts['en']['select_language'],
            reply_markup=get_language_keyboard()
        )
        return
    
    user_lang = user_languages[user_id]
    
    # Remove from current chat
    if user_id in users:
        partner_id = users[user_id]
        if partner_id in users:
            del users[partner_id]
            await bot.send_message(partner_id, get_text(partner_id, 'partner_left'))
        del users[user_id]
    
    # Remove from all queues first
    for lang_queue in queues.values():
        if user_id in lang_queue:
            lang_queue.remove(user_id)
    
    # Add to appropriate language queue
    if user_id not in queues[user_lang]:
        queues[user_lang].append(user_id)
    
    await find_partner(user_id)

@dp.message(lambda message: message.text == "/stop")
async def stop_chat(message: types.Message):
    user_id = message.from_user.id
    
    if user_id not in user_languages:
        await message.answer(
            texts['en']['select_language'],
            reply_markup=get_language_keyboard()
        )
        return
    
    if user_id in users:
        partner_id = users[user_id]
        del users[user_id]
        if partner_id in users:
            del users[partner_id]
            await bot.send_message(partner_id, get_text(partner_id, 'partner_left'))
        await message.answer(get_text(user_id, 'chat_stopped'))
    else:
        await message.answer(get_text(user_id, 'not_in_chat'))

async def find_partner(user_id):
    user_lang = user_languages[user_id]
    
    if len(queues[user_lang]) >= 2:
        queues[user_lang].remove(user_id)
        partner_id = queues[user_lang].pop(0)
        
        # Connect users
        users[user_id] = partner_id
        users[partner_id] = user_id
        
        await bot.send_message(user_id, get_text(user_id, 'connected'))
        await bot.send_message(partner_id, get_text(partner_id, 'connected'))
    else:
        await bot.send_message(user_id, get_text(user_id, 'searching'))

@dp.message()
async def handle_message(message: types.Message):
    user_id = message.from_user.id
    
    # Check if user has selected language
    if user_id not in user_languages:
        await message.answer(
            texts['en']['select_language'],
            reply_markup=get_language_keyboard()
        )
        return
    
    # If user has partner, forward message
    if user_id in users:
        partner_id = users[user_id]
        
        try:
            if message.text:
                await bot.send_message(partner_id, message.text)
            elif message.photo:
                await bot.send_photo(partner_id, message.photo[-1].file_id, caption=message.caption)
            elif message.video:
                await bot.send_video(partner_id, message.video.file_id, caption=message.caption)
            elif message.voice:
                await bot.send_voice(partner_id, message.voice.file_id)
            elif message.sticker:
                await bot.send_sticker(partner_id, message.sticker.file_id)
            elif message.document:
                await bot.send_document(partner_id, message.document.file_id, caption=message.caption)
            elif message.audio:
                await bot.send_audio(partner_id, message.audio.file_id, caption=message.caption)
        except Exception as e:
            print(f"Error forwarding message: {e}")
            # Partner blocked bot or deleted account
            if user_id in users:
                del users[user_id]
            await message.answer(get_text(user_id, 'partner_left'))
    else:
        # Find partner for user
        user_lang = user_languages[user_id]
        
        # Remove from all queues first
        for lang_queue in queues.values():
            if user_id in lang_queue:
                lang_queue.remove(user_id)
        
        # Add to appropriate language queue
        if user_id not in queues[user_lang]:
            queues[user_lang].append(user_id)
        
        await find_partner(user_id)

async def main():
    # Database setup
    try:
        conn = await get_db()
        await conn.execute('''
            CREATE TABLE IF NOT EXISTS users (
                user_id BIGINT PRIMARY KEY,
                username TEXT,
                first_name TEXT,
                language TEXT NOT NULL DEFAULT 'en',
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        await conn.close()
        print("✅ Database tables created/verified")
    except Exception as e:
        print(f"❌ Database error: {e}")
        print("⚠️ Bot will work without database features")
    
    print("🚀 Multi-language Anonymous Chat Bot Started!")
    print("📊 Supported languages: 🇺🇿 O'zbek, 🇷🇺 Русский, 🇺🇸 English")
    await dp.start_polling(bot)

if __name__ == "__main__":
    asyncio.run(main())

ModuleNotFoundError: No module named 'aiogram'

In [13]:
import cv2
import numpy as np

def main():
    # Kamera yoki video faylni ochish
    cap = cv2.VideoCapture(0)  # 0 - birinchi kamera, fayl uchun fayl yo'lini bering
    
    # Haar cascade classifier yuklab olish (yuz aniqlash uchun)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Kamera ochilishini tekshirish
    if not cap.isOpened():
        print("Xato: Kamerani ochib bo'lmadi!")
        return
    
    print("Dastur ishga tushdi. 'q' tugmasini bosing chiqish uchun.")
    
    while True:
        # Kadrni o'qish
        ret, frame = cap.read()
        
        if not ret:
            print("Kadr o'qib bo'lmadi!")
            break
        
        # Kadrni oyna o'lchamiga moslash
        frame = cv2.resize(frame, (800, 600))
        
        # Kulrang rangga o'tkazish (yuz aniqlash uchun)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Yuzlarni aniqlash
        faces = face_cascade.detectMultiScale(
            gray,
            scaleFactor=1.1,        # Har bir darajada tasvirning qanchalik kichraytirilishi
            minNeighbors=5,         # Har bir nomzod to'rtburchak qancha qo'shni to'rtburchaklarga ega bo'lishi kerak
            minSize=(30, 30),       # Minimal yuz o'lchami
            flags=cv2.CASCADE_SCALE_IMAGE
        )
        
        # Har bir aniqlangan yuz uchun
        for (x, y, w, h) in faces:
            # Yuz mintaqasini ajratib olish
            face_roi = frame[y:y+h, x:x+w]
            
            # Kuchli blur effekti qo'llash
            # Gaussian blur yoki motion blur ishlatishingiz mumkin
            blurred_face = cv2.GaussianBlur(face_roi, (99, 99), 30)
            
            # Alternativ: Motion blur
            # kernel_size = 50
            # motion_blur_kernel = np.zeros((kernel_size, kernel_size))
            # motion_blur_kernel[int((kernel_size-1)/2), :] = np.ones(kernel_size)
            # motion_blur_kernel = motion_blur_kernel / kernel_size
            # blurred_face = cv2.filter2D(face_roi, -1, motion_blur_kernel)
            
            # Blur qilingan yuzni asl kadrga qaytarish
            frame[y:y+h, x:x+w] = blurred_face
            
            # Yuz atrofida to'rtburchak chizish (ixtiyoriy)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            # Matn qo'shish (ixtiyoriy)
            cv2.putText(frame, 'Yuz aniqlandi', (x, y-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        # Natijani ko'rsatish
        cv2.imshow('Yuz Aniqlash va Blur', frame)
        
        # 'q' tugmasi bosilganda chiqish
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Resurslarni tozalash
    cap.release()
    cv2.destroyAllWindows()

# Dasturni ishga tushirish
if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'cv2'